In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import psycopg2
import datetime
import plotly.io as pio
import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import gc
from os import walk
import geopandas as gpd
import math

In [3]:
cnt = 0
f = r'./result_new/features_percentile005'
path_list = os.listdir(f)
path_list.sort(key=lambda x:int(x[15:-4]))
for file in path_list:
    filename = f + '/' + file
    if cnt == 0:                               
        df_edge = pd.read_csv(filename,skiprows=[1])
    else:
        df = pd.read_csv(filename,skiprows=[1])
        df_edge = pd.concat([df_edge,df])
    cnt += 1
df_edge = df_edge.reset_index(drop = True)

In [4]:
df_edge.describe()

,network_id,position,average_speed,speed_limit,mass,elevation_change,previous_orientation,length,energy_consumption_total,energy_consumption_per_hour,energy_consumption_per_100km,time,direction_angle,time_stage,week_day,lanes,bridge,endpoint_u,endpoint_v
count,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120376.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000,120475.000000
mean,397608.653372,81.111135,77.825436,80.261621,23168.059763,-0.113145,-1.856240,632.578962,0.223233,0.000745,58.905575,30.069145,1.667142,3.269151,3.074588,1.864619,0.192496,1.200274,1.157136
std,118314.595349,77.941162,31.247457,21.895262,8245.075889,10.699713,34.860204,1024.806320,0.375075,0.000477,282.952868,42.159121,102.804529,0.823000,1.288722,1.359442,0.394262,1.781827,1.764738
min,29172.000000,1.000000,2.113339,24.135000,8000.000000,-260.500000,-179.999959,1.290000,0.000000,0.000000,0.000000,0.100000,-180.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,368537.000000,26.000000,52.906972,48.270000,16000.000000,-4.550000,-0.818007,173.783000,0.050372,0.000367,20.622006,10.400000,-89.424764,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,448990.000000,56.000000,91.497178,88.495000,20000.000000,0.000000,-0.001152,410.534000,0.133239,0.000708,32.301434,21.000000,0.309616,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000
75%,455733.500000,107.000000,101.869484,96.540000,30000.000000,4.500000,1.044766,778.761000,0.268656,0.001037,48.950687,33.000000,90.358604,4.000000,4.000000,3.000000,0.000000,4.000000,4.000000
max,716846.000000,495.000000,254.000000,112.630000,49000.000000,212.500000,179.999490,15022.949000,7.459303,0.002618,43519.675060,850.100000,179.997073,6.000000,7.000000,6.000000,1.000000,13.000000,13.000000


In [5]:
df_edge.head()

,network_id,osmid,tags,index,trip_id,position,road_type,average_speed,speed_limit,mass,...,speed,direction,direction_angle,time_stage,week_day,time_acc,lanes,bridge,endpoint_u,endpoint_v
0,408526,"[636154193, 6012871]","(34504451, 33826567, 0)","(1650, 13590)","(0, 1)",1,tertiary,6.367262,56.315,8000.0,...,"[3.4102, 3.4648, 3.4258, 3.4805, 3.4922, 3.507...","[0.00023500000000353793, 0.0004609999999871661]",62.989269,2,1,2020-08-10 06:02:00,2,0,0,0
1,391523,"[636154193, 5998138]","(33826567, 33308886, 0)","(13590, 13740)","(0, 1)",2,tertiary,21.977953,56.315,30000.0,...,"[19.188, 20.281, 20.777, 21.914, 22.707, 23.00...","[0.0007799999999988927, 2.500000000793534e-05]",1.835775,2,1,2020-08-10 06:48:26.100000,2,0,0,6
2,347664,"[636154193, 5998138]","(33308886, 33826567, 0)","(13740, 14070)","(0, 1)",3,tertiary,15.799874,56.315,30000.0,...,"[21.121, 20.887, 20.656, 20.469, 20.266, 20.02...","[0.0009549999999975967, 1.9999999949504854e-06]",0.119991,2,1,2020-08-10 06:48:41.100000,2,0,6,0
3,391522,"[636154193, 6012871]","(33826567, 34504451, 0)","(14070, 14790)","(0, 1)",4,tertiary,23.135569,56.315,30000.0,...,"[1.793, 4.543, 6.1602, 7.3906, 7.8281, 8.375, ...","[-0.0019369999999980791, 0.0032980000000009113]",120.426809,2,1,2020-08-10 06:49:14.100000,2,0,0,0
4,477837,6012871,"(6960167568, 375477545, 0)","(14790, 15090)","(0, 1)",5,tertiary,45.601827,56.315,30000.0,...,"[40.59, 40.855, 41.484, 41.785, 41.922, 42.09,...","[-0.00288900000000325, 0.0024920000000037135]",139.219529,2,1,2020-08-10 06:50:26.100000,2,0,0,0


In [6]:
df_edge['osmNodeIdUV'] = df_edge.tags.apply(lambda x: tuple(list(map(int, x[1:-1].split(", ")))[:-1]))

In [7]:
df_edge.head()

,network_id,osmid,tags,index,trip_id,position,road_type,average_speed,speed_limit,mass,...,direction,direction_angle,time_stage,week_day,time_acc,lanes,bridge,endpoint_u,endpoint_v,osmNodeIdUV
0,408526,"[636154193, 6012871]","(34504451, 33826567, 0)","(1650, 13590)","(0, 1)",1,tertiary,6.367262,56.315,8000.0,...,"[0.00023500000000353793, 0.0004609999999871661]",62.989269,2,1,2020-08-10 06:02:00,2,0,0,0,"(34504451, 33826567)"
1,391523,"[636154193, 5998138]","(33826567, 33308886, 0)","(13590, 13740)","(0, 1)",2,tertiary,21.977953,56.315,30000.0,...,"[0.0007799999999988927, 2.500000000793534e-05]",1.835775,2,1,2020-08-10 06:48:26.100000,2,0,0,6,"(33826567, 33308886)"
2,347664,"[636154193, 5998138]","(33308886, 33826567, 0)","(13740, 14070)","(0, 1)",3,tertiary,15.799874,56.315,30000.0,...,"[0.0009549999999975967, 1.9999999949504854e-06]",0.119991,2,1,2020-08-10 06:48:41.100000,2,0,6,0,"(33308886, 33826567)"
3,391522,"[636154193, 6012871]","(33826567, 34504451, 0)","(14070, 14790)","(0, 1)",4,tertiary,23.135569,56.315,30000.0,...,"[-0.0019369999999980791, 0.0032980000000009113]",120.426809,2,1,2020-08-10 06:49:14.100000,2,0,0,0,"(33826567, 34504451)"
4,477837,6012871,"(6960167568, 375477545, 0)","(14790, 15090)","(0, 1)",5,tertiary,45.601827,56.315,30000.0,...,"[-0.00288900000000325, 0.0024920000000037135]",139.219529,2,1,2020-08-10 06:50:26.100000,2,0,0,0,"(6960167568, 375477545)"


In [10]:
segmentsElevation = dict()
for i in range(len(df_edge)):
    osmNode = df_edge.loc[i,'osmNodeIdUV']
    reverseNode = osmNode[::-1]
    if osmNode not in segmentsElevation:
        segmentsElevation[osmNode] = df_edge.loc[i,'elevation_change']
    if reverseNode not in segmentsElevation:
        segmentsElevation[reverseNode] = -df_edge.loc[i,'elevation_change']

In [11]:
len(segmentsElevation)

16321

In [13]:
segmentsElevation[(33295092, 33800025)]

0.6000000000000227

In [14]:
import csv
output_root = "segmentElevationChange.csv"
csvFile = open(output_root, "w")
writer = csv.writer(csvFile)
writer.writerow(["uvPair", "elevation change"])
for i in segmentsElevation:
    writer.writerow([i, segmentsElevation[i]])
csvFile.close()
np.save('segmentElevationChange.npy', segmentsElevation)